In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import date, timedelta

In [161]:
tuttoarr2 = []
r  = requests.get("http://az.lib.ru/d/dostoewskij_f_m/",   )
data = r.text
soup = BeautifulSoup(data)
inputTag = soup.findAll("dl")



In [162]:
for link in inputTag:               
    try:                        
        title = link.findChildren("a",recursive=True)
        for titlea in title: 
            if titlea.has_attr('href') and not titlea.has_attr('class'):
                if 'comment' not in titlea['href']:
                    if 'shtml' in titlea['href']:
                        tuttoarr2.append(tuple((titlea['href'],titlea.text)))
                    
    except IndexError:
        print('Rien')   

In [ ]:
newarr={'title':[],'text':[]}
i =0 
for elem in tuttoarr2:
    r  = requests.get("http://az.lib.ru/d/dostoewskij_f_m/" + elem[0]   )
    data = r.text
    soup2 = BeautifulSoup(data)
    inputTag2 = soup2.findAll("div", {"align":"justify"})
    for link in inputTag2:
        if i>5:
            break
        print(i)
        i+=1
        try:
            title = link.findAll("dd",recursive=True)
            for titlea in title: 
                if titlea.text not in newarr['text']:
                    newarr['title'].append(elem[1])
                    newarr['text'].append(titlea.text)

        except IndexError:
            print('Rien') 

0
1
2
3
4
5


In [ ]:
import pandas as pd
det = pd.DataFrame(columns=['title','text'])
det['title'] = newarr['title']
det['text'] = newarr['text']
det.to_csv('dostoewsky.csv', index=False)

In [ ]:
data = pd.read_csv('dostoewsky.csv')

In [ ]:
data

In [93]:
data2 = pd.read_csv('pushkin.csv')

In [79]:
from nltk.tokenize import TweetTokenizer
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

In [80]:
tokenizer = TweetTokenizer()
def preprocess(text):
    words = tokenizer.tokenize(text)
    return ' '.join(words).lower()

In [104]:
data['prepr'] = data.text.apply(preprocess)

In [94]:
data2['prepr'] = data2.text.apply(preprocess)

In [83]:
vectorizer = TfidfVectorizer()

In [106]:
fitted = pd.concat([data.prepr,data2.prepr], axis =0)

In [109]:
vectorizer.fit(fitted)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [110]:
vectorized = vectorizer.transform(data.prepr) 

In [111]:
vectorized2 = vectorizer.transform(data2.prepr)

In [126]:
vectorized.shape

(1694, 23492)

In [129]:
vect = vectorized2.copy()

In [135]:
vectorized2.resize(1694,23492)

In [136]:
vectorized2[:816,:23492] = vect

ValueError: shape mismatch in assignment

In [134]:
vectorized2.shape

(816, 23492)

In [120]:
import numpy as np

In [132]:
result = np.array(vectorized) - np.array(vectorized2)

ValueError: inconsistent shapes